Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

• Наименование вакансии.

• Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).

• Ссылку на саму вакансию.

• Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint

In [2]:
def hh(vacancy, page_sum):
    
    jobs_list = []
    df = []
 
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36'}
    base_url = 'https://hh.ru'
    url = 'https://hh.ru/search/vacancy?clusters=true&area=1&ored_clusters=true&enable_snippets=true&salary=&text=' + vacancy
    i = 1
    
    while i <= page_sum:
        response = requests.get(url, headers=headers)
        if response.ok:
            dom = BeautifulSoup(response.text, 'html.parser')
            jobs_block = dom.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})
        
        # название вакансии
            for job in jobs_block:
                jobs_data = {}
                job_text = job.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
                vacancy_name = job_text.getText()
                jobs_data['vacancy_name'] = vacancy_name
    
    
        #зп
                salary = job.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
                if not salary:
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    salary = salary.getText().replace(u'\xa0', u'')
                    salary = re.split(r'\s|[-–, ]', salary)

                    if salary[0] == 'до':
                        salary_min = None
                        if salary[2] == '000':
                            salary_max = salary[1] + salary[2]
                            salary_max = int(salary_max)
                    elif salary[0] == 'от':
                        salary_max = None
                        if salary[2] == '000':
                            salary_min = salary[1] + salary[2]
                            salary_min = int(salary_min)
                    else:
                        salary_min = salary[1] + salary[2]
                        salary_min = int(salary_min)
                        salary_max = salary[3] + salary[4]
                        salary_max = int(salary_max)
                    salary_currency = salary[-1]
                jobs_data['salary_min'] = salary_min
                jobs_data['salary_max'] = salary_max
                jobs_data['salary_currency'] = salary_currency
                
        #ссылка на вакансию
                link_text = job.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
                link = link_text['href']
                jobs_data['link'] = link
                jobs_data['site'] = base_url
                                

                jobs_list.append(jobs_data)
           
            page_text = dom.find('a', {'data-qa': 'pager-next'})
            link_1 = page_text['href']
            
        url = base_url + link_1

        i = i + 1
    

    df = pd.DataFrame(jobs_list)
    return df
    print(df.head())

In [3]:
hh('Python', 2)

,vacancy_name,salary_min,salary_max,salary_currency,link,site
0,Специалист по тестированию,NaN,NaN,None,https://krasnodar.hh.ru/vacancy/40585500?from=...,https://hh.ru
1,Разработчик C/C++,NaN,220000.0,руб.,https://krasnodar.hh.ru/vacancy/51998774?from=...,https://hh.ru
2,Middle Analyst,NaN,NaN,None,https://krasnodar.hh.ru/vacancy/52430372?from=...,https://hh.ru
3,"Data Analyst \ Аналитик данных (BI, SQL, Pytho...",0.0,210.0,руб.,https://krasnodar.hh.ru/vacancy/49881796?from=...,https://hh.ru
4,QA специалист в команду Биллинга,NaN,NaN,None,https://krasnodar.hh.ru/vacancy/47251783?from=...,https://hh.ru
5,Data Scientist (команда Поиска),NaN,NaN,None,https://krasnodar.hh.ru/vacancy/46048820?from=...,https://hh.ru
6,Devops engineer (Hadoop),300000.0,NaN,руб.,https://krasnodar.hh.ru/vacancy/49493874?from=...,https://hh.ru
7,"Ведущий аналитик (Machine Learning, SAS)",NaN,NaN,None,https://krasnodar.hh.ru/vacancy/52433607?from=...,https://hh.ru
8,Аналитик данных,NaN,NaN,None,https://krasnodar.hh.ru/vacancy/46241509?from=...,https://hh.ru
9,Junior разработчик Python Django,0.0,80.0,руб.,https://krasnodar.hh.ru/vacancy/52203762?from=...,https://hh.ru


In [4]:
hh('C', 3)

,vacancy_name,salary_min,salary_max,salary_currency,link,site
0,Разработчик C/C++,NaN,220000.0,руб.,https://krasnodar.hh.ru/vacancy/51998774?from=...,https://hh.ru
1,Мобильный разработчик IOS,200000.0,NaN,руб.,https://krasnodar.hh.ru/vacancy/52246529?from=...,https://hh.ru
2,"Разработчик Unreal Engine, С++",0.0,350.0,руб.,https://krasnodar.hh.ru/vacancy/52263943?from=...,https://hh.ru
3,Программист со знанием C/C++ (удалённо),120000.0,NaN,руб.,https://krasnodar.hh.ru/vacancy/52308044?from=...,https://hh.ru
4,"Backend developer. Middle+/Senior. Go, Python,...",0.0,6.0,USD,https://krasnodar.hh.ru/vacancy/52247182?from=...,https://hh.ru
5,Разработчик Computer Vision,250000.0,NaN,руб.,https://krasnodar.hh.ru/vacancy/52333047?from=...,https://hh.ru
6,Программист C (embedded),NaN,200000.0,руб.,https://krasnodar.hh.ru/vacancy/52107339?from=...,https://hh.ru
7,Разработчик С++,170000.0,NaN,руб.,https://krasnodar.hh.ru/vacancy/51949832?from=...,https://hh.ru
8,Senior C#/ .NET Developer,0.0,8.0,USD,https://krasnodar.hh.ru/vacancy/52422702?from=...,https://hh.ru
9,Разработчик С/С++,NaN,300000.0,руб.,https://krasnodar.hh.ru/vacancy/52235534?from=...,https://hh.ru
